In [2]:
# Imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Data
BTVpolice = pd.read_csv('data/BTVpolice_cleaned.csv')

## Try mapping by different factors

In [ ]:
# Date
#Think about holidays/significant events
#Find outliers

BTVpolice['call_time'] = pd.to_datetime(BTVpolice['call_time'])
BTVpolice['date'] = BTVpolice['call_time'].dt.date
calls_per_day = BTVpolice.groupby('date').size()
mean_calls = calls_per_day.mean()
std_calls = calls_per_day.std()
outliers = calls_per_day[calls_per_day > (mean_calls + 2 * std_calls)]

print(outliers)


date
2023-04-14    105
2023-06-02    107
2023-07-12    131
2023-07-14    135
2023-07-17    113
2023-07-19    128
2023-07-20    128
2023-07-21    133
2023-07-28    120
2023-08-07    106
2023-08-10    107
2023-09-01    115
2023-09-06    108
2023-09-07    128
2023-09-08    127
2023-09-16    105
2023-09-22    111
2023-10-05    118
2023-10-06    118
2023-10-13    117
2023-10-19    115
2023-10-28    111
2024-05-06    105
2024-05-18    109
2024-05-20    105
2024-08-14    117
dtype: int64


In [ ]:
#Priority

In [ ]:
#Drug-related, dv-related, alcohol-related, mental-health related

In [ ]:
# Call_type

In [ ]:
#Hour (histogram)

In [ ]:
#Day of week (histogram)

In [ ]:
#Month (histogram)